In [1]:
using MLJ
using ScikitLearn
using Random, Statistics
using CSV
using DataFrames

In [2]:
# read the CSV file
df = CSV.read("hikari.csv", DataFrame)

df = select(df, [:"idle.tot", :"flow_iat.tot", :"fwd_iat.tot", :"active.avg", :"idle.max", :"flow_iat.max", :"fwd_iat.max", :"active.max", :"idle.avg", :"active.tot", :"bwd_iat.max", :"idle.min", :"flow_iat.avg", :"fwd_iat.avg", :"flow_iat.min", :"fwd_iat.min", :"fwd_iat.std", :"bwd_iat.std", :"flow_iat.std", :"bwd_iat.avg", :"Label"])



train_ratio = 0.8
test_ratio = 1.0 - train_ratio

nrows = nrow(df)
train_indices = randsubseq(1:nrows, train_ratio)
test_indices = setdiff(1:nrows, train_indices)

train_df = df[train_indices, :]
test_df = df[test_indices, :]

Row,idle.tot,flow_iat.tot,fwd_iat.tot,active.avg,idle.max,flow_iat.max,fwd_iat.max,active.max,idle.avg,active.tot,bwd_iat.max,idle.min,flow_iat.avg,fwd_iat.avg,flow_iat.min,fwd_iat.min,fwd_iat.std,bwd_iat.std,flow_iat.std,bwd_iat.avg,Label
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64
1,7.54531e6,7.78062e6,7.78062e6,1.17658e5,7.54531e6,7.54531e6,7.54531e6,2.06765e5,7.54531e6,2.35315e5,7.61372e6,7.54531e6,2.88171e5,5.98509e5,9.05991,16.9277,2.08742e6,2.10853e6,1.45041e6,5.96219e5,1
2,8.88404e6,9.11743e6,9.11743e6,116696.0,8.88404e6,8.88404e6,8.88404e6,204406.0,8.88404e6,233392.0,8.95262e6,8.88404e6,3.5067e5,7.01341e5,9.05991,20.0272,2.45873e6,2.58086e6,1.74052e6,7.5739e5,1
3,0.0,4.69255e6,4.69255e6,4.69255e6,0.0,4.17538e6,4.17538e6,4.69255e6,0.0,4.69255e6,4.24396e6,0.0,1.67591e5,3.35182e5,10.9673,15.0204,1.10783e6,1.16988e6,7.87287e5,3.58704e5,1
4,0.0,2.1987e6,2.1987e6,2.1987e6,0.0,1.96554e6,1.96554e6,2.1987e6,0.0,2.1987e6,2.03487e6,0.0,81433.2,1.69131e5,18.8351,36.0012,5.40368e5,5.61415e5,3.76764e5,1.66843e5,1
5,1.37657e7,1.40043e7,1.40043e7,1.19272e5,1.37657e7,1.37657e7,1.37657e7,208838.0,1.37657e7,2.38545e5,1.38346e7,1.37657e7,5.38626e5,1.07725e6,6.91414,19.7887,3.8125e6,3.99004e6,2.69784e6,1.16455e6,1
6,0.0,4.63287e6,4.63287e6,4.63287e6,0.0,4.40288e6,4.40288e6,4.63287e6,0.0,4.63287e6,4.47095e6,0.0,1.78187e5,3.56375e5,6.91414,20.0272,1.21609e6,1.28725e6,8.6177e5,3.83579e5,1
7,1.32509e7,1.37232e7,1.37232e7,2.36145e5,1.32509e7,1.32509e7,1.32509e7,4.4337e5,1.32509e7,472291.0,1.33194e7,1.32509e7,5.08268e5,1.05563e6,13.113,40.0543,3.66504e6,3.68609e6,2.54709e6,1.05335e6,1
8,6.63656e6,6.86676e6,6.86676e6,115099.0,6.63656e6,6.63656e6,6.63656e6,201714.0,6.63656e6,2.30198e5,6.70497e6,6.63656e6,2.64106e5,5.28212e5,7.15256,20.0272,1.83551e6,1.93213e6,1.2998e6,569753.0,1
9,1.57289e7,1.59619e7,1.59619e7,1.16505e5,1.57289e7,1.57289e7,1.57289e7,2.02999e5,1.57289e7,2.33009e5,1.57982e7,1.57289e7,5.9118e5,1.22784e6,11.2057,20.0272,4.35709e6,4.37854e6,3.02532e6,1.2256e6,1


In [8]:
using ScikitLearn
@sk_import linear_model:LogisticRegression

PyObject <class 'sklearn.linear_model._logistic.LogisticRegression'>

In [4]:
# converting into vectors
Ada_X_train = Matrix(select(train_df, Not(:Label)))

Ada_y_train = Vector(train_df.Label)

Ada_X_test = Matrix(select(test_df, Not(:Label)))

Ada_y_test = Vector(test_df.Label)

111011-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 1
 1
 1
 1
 1
 1
 1
 1
 1

In [20]:
# Create an AdaBoostClassifier
ada_model = LogisticRegression(max_iter=150)

PyObject LogisticRegression(max_iter=150)

In [21]:
# Fit the model to your data
ScikitLearn.fit!(ada_model, Ada_X_train, Ada_y_train)

/Users/davuluri_hemanth/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


PyObject LogisticRegression(max_iter=150)

In [22]:
# Make predictions
ada_y_pred = ScikitLearn.predict(ada_model, Ada_X_test)


#accuracy
accuracy = mean(ada_y_pred .== Ada_y_test)
# Calculate True Positives (TP), True Negatives (TN), False Positives (FP), and False Negatives (FN)
TP = sum((ada_y_pred .== 1) .& (Ada_y_test .== 1))
TN = sum((ada_y_pred .== 0) .& (Ada_y_test .== 0))
FP = sum((ada_y_pred .== 1) .& (Ada_y_test .== 0))
FN = sum((ada_y_pred .== 0) .& (Ada_y_test .== 1))

# Calculate precision, recall, and F1-score
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)

println("Accuracy: $accuracy")
println("Precision: $precision")
println("Recall: $recall")
println("F1-Score: $f1_score")

Accuracy: 0.908612660006666
Precision: 0.16060862214708369
Recall: 0.07368148914167529
F1-Score: 0.10101905183872396
